# IA Jojo

## Introdução

## Sumário

1. [Instalação de Bibliotecas](#Instalação-de-Bibliotecas)
2. [Pré processamento de Imagens](#Pré-processamento-de-Imagens)
3. [Escolha do modelo](#Escolha-do-Modelo)
4. [Treinamento do Modelo](#Treinamento-do-Modelo)
5. [Avaliação do Modelo](#Avaliação-do-Modelo)

## Instalação de Bibliotecas

Primeiramente, se ainda não foi feito, instalaremos todas as bibliotecas necessárias para esse projeto:


In [ ]:
%pip install -r requirements.txt

## Pré-processamento de Imagens


In [ ]:
from pathlib import Path
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definindo os caminhos
CHAR_FOLDER = Path("Characters")
TEST_FOLDER = CHAR_FOLDER / "Test"
TRAIN_FOLDER = CHAR_FOLDER / "Train"

# Transformações para pré-processamento das imagens
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Carregando os datasets
image_datasets = {
    'train': datasets.ImageFolder(TRAIN_FOLDER, transform=data_transforms['train']),
    'test': datasets.ImageFolder(TEST_FOLDER, transform=data_transforms['test'])
}

# Criando os data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4)
}

# Obtendo o número de classes
class_names = image_datasets['train'].classes
num_classes = len(class_names)

print(f"Classes: {class_names}")

## Escolha do Modelo

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Carregando o modelo VGG16 pré-treinado
model = models.vgg16(pretrained=True)

# Congelando os parâmetros do modelo
for param in model.parameters():
    param.requires_grad = False

# Modificando a última camada
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, num_classes)

print(model)

## Treinamento do Modelo

In [ ]:
import torch.optim as optim

# Definindo a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)

# Treinamento do modelo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Treinamento
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders['train'].dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validação
        model.eval()   # Set model to evaluate mode
        running_loss = 0.0
        running_corrects = 0

        with torch.no_grad():
            for inputs, labels in dataloaders['test']:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders['test'].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders['test'].dataset)

        print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

model = train_model(model, dataloaders, criterion, optimizer, num_epochs=25)

## Avaliação do Modelo

In [ ]:
def evaluate_model(model, dataloaders):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloaders['test'].dataset)
    print(f'Test Accuracy: {accuracy:.4f}')

evaluate_model(model, dataloaders)